In [1]:
import pandas as pd
import pydot
import numpy as np
from gensim.models import TfidfModel
import src.situ_helper as situ
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv(
    'data/interim/situ_topics_kmeans_tfidf.csv',
    converters = {'sop': eval, 'situ_lst': eval}
)
df['sop_graph'] = df['sop'].apply(lambda x: '\n'.join(x))
df = df[['type', 'juri', 'role', 'filename', 'situation', 'sop_graph', 'cluster', 'situ_lst', 'sop']]
df2 = df.drop(columns = ['role', 'filename', 'juri'])
# df2

In [3]:
df3 = df2[df2['cluster'] == 34].copy().reset_index(drop = True)
df3#[df3['type'] == 'BYLAW']

,type,situation,sop_graph,cluster,situ_lst,sop
0,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am...",34,[Missing person located in Abbotsford and repo...,"[Refer to AB - ASSPFA - Assist police, fire, a..."
1,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am...",34,[Missing person sighted],"[Refer to AB - ASSPFA - Assist police, fire, a..."
2,MISSIP,Missing person located on Bowen Island and rep...,"Refer to BI - ASSPFA - Assist police, fire, am...",34,[Missing person located on Bowen Island and re...,"[Refer to BI - ASSPFA - Assist police, fire, a..."
3,MISSIP,Missing person sighted,"Refer to BI - ASSPFA - Assist police, fire, am...",34,[Missing person sighted],"[Refer to BI - ASSPFA - Assist police, fire, a..."
4,MISSIP,Missing person located in Burnaby and report i...,"Refer to BU - ASSPFA - Assist police, fire, am...",34,[Missing person located in Burnaby and report ...,"[Refer to BU - ASSPFA - Assist police, fire, a..."
5,MISSIP,Missing person sighted,"Refer to BU - ASSPFA - Assist police, fire, am...",34,[Missing person sighted],"[Refer to BU - ASSPFA - Assist police, fire, a..."
6,MISSIP,Missing person located in Delta and report is ...,"Refer to DE - ASSPFA - Assist police, fire, am...",34,[Missing person located in Delta and report is...,"[Refer to DE - ASSPFA - Assist police, fire, a..."
7,MISSIP,Missing person sighted,"Refer to DE - ASSPFA - Assist police, fire, am...",34,[Missing person sighted],"[Refer to DE - ASSPFA - Assist police, fire, a..."
8,MISSIP,Missing person located in New Westminster and ...,"Refer to NW - ASSPFA - Assist police, fire, am...",34,[Missing person located in New Westminster and...,"[Refer to NW - ASSPFA - Assist police, fire, a..."
9,MISSIP,Missing person sighted,"Refer to NW - ASSPFA - Assist police, fire, am...",34,[Missing person sighted],"[Refer to NW - ASSPFA - Assist police, fire, a..."


In [4]:
situ_tfidf = TfidfVectorizer().fit(df3['situ_lst'].apply(situ.preprocess))
situ_tfidf_mtx = situ_tfidf.transform(df3['situation'])

sop_tfidf =  TfidfVectorizer().fit(df3['sop'].apply(situ.preprocess))
sop_tfidf_mtx = sop_tfidf.transform(df3['sop_graph'])

In [5]:
situ_similarity = cosine_similarity(situ_tfidf_mtx)
sop_similarity = cosine_similarity(sop_tfidf_mtx)

In [6]:
situ_similarity[0]

array([1.        , 0.22078566, 0.24506967, 0.22078566, 0.31784981,
       0.22078566, 0.31784981, 0.22078566, 0.24506967, 0.22078566,
       0.24506967, 0.22078566, 0.31784981, 0.22078566, 0.31784981,
       0.22078566, 0.24506967, 0.22078566, 0.31784981, 0.22078566,
       0.20671435, 0.22078566, 0.38185094, 0.22078566, 0.34271854,
       0.22078566, 0.24506967, 0.22078566, 0.25594051, 0.22078566])

In [7]:
df3['situ_group'] = df3['situation']
df3['sop_group'] = df3['sop_graph']
similarity_threshold = 0.245
situ_grouped = set()
for i in range(situ_similarity.shape[0]):
    if i in situ_grouped:
        continue
    same_group = np.where(situ_similarity[i] >= similarity_threshold)[0]
    situ_grouped = situ_grouped | set(same_group)
    for j in same_group:
        df3.at[int(j), 'situ_group'] = df3.iloc[i]['situation']
sop_grouped = set()
for i in range(sop_similarity.shape[0]):
    if i in sop_grouped:
        continue
    same_group = np.where(sop_similarity[i] >= similarity_threshold)[0]
    sop_grouped = sop_grouped | set(same_group)
    for j in same_group:
        df3.at[j, 'sop_group'] = df3.iloc[i]['sop_graph']
df3[['type', 'situ_group', 'sop_group']]

,type,situ_group,sop_group
0,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am..."
1,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am..."
2,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am..."
3,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am..."
4,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am..."
5,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am..."
6,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am..."
7,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am..."
8,MISSIP,Missing person located in Abbotsford and repor...,"Refer to AB - ASSPFA - Assist police, fire, am..."
9,MISSIP,Missing person sighted,"Refer to AB - ASSPFA - Assist police, fire, am..."


In [8]:
for i, df in df2.groupby('cluster'):
    df3 = df.copy().reset_index(drop = True)
    cluster = int(df3.iloc[0]['cluster'])

    situ_tfidf = TfidfVectorizer().fit(df3['situ_lst'].apply(situ.preprocess))
    situ_tfidf_mtx = situ_tfidf.transform(df3['situation'])
    sop_tfidf =  TfidfVectorizer().fit(df3['sop'].apply(situ.preprocess))
    sop_tfidf_mtx = sop_tfidf.transform(df3['sop_graph'])
    situ_similarity = cosine_similarity(situ_tfidf_mtx)
    sop_similarity = cosine_similarity(sop_tfidf_mtx)

    df3['situ_group'] = df3['situation']
    df3['sop_group'] = df3['sop_graph']

    similarity_threshold = 0.45
    situ_grouped = set()
    for i in range(situ_similarity.shape[0]):
        if i in situ_grouped:
            continue
        same_group = np.where(situ_similarity[i] >= similarity_threshold)[0]
        situ_grouped = situ_grouped | set(same_group)
        for j in same_group:
            df3.at[j, 'situ_group'] = df3.iloc[i]['situation']

    sop_grouped = set()
    for i in range(sop_similarity.shape[0]):
        if i in sop_grouped:
            continue
        same_group = np.where(sop_similarity[i] >= similarity_threshold)[0]
        sop_grouped = sop_grouped | set(same_group)
        for j in same_group:
            df3.at[j, 'sop_group'] = df3.iloc[i]['sop_graph']

    graph = pydot.Dot(graph_type='graph', concentrate=True, rankdir='LR')
    df3 = df3[['type', 'situ_group', 'sop_group']]
    nrows, _ = df3.shape
    for i in range(nrows):
        row = df3.iloc[i]
        for p, c in zip(row[:-1], row[1:]):
            edge = pydot.Edge(p, c)
            graph.add_edge(edge)
    graph.write_png(f"img/flowcharts/situ_cluster_{cluster}.png")

In [9]:
raise Exception('Stop here')

Exception: Stop here

In [ ]:
# for i, df in df2.groupby('cluster'):
#     graph = pydot.Dot(graph_type='graph', rankdir='LR')
#     cluster = int(df.iloc[0]['cluster'])
#     df = df.drop(columns = 'cluster')
#     nrows, _ = df.shape
#     for i in range(nrows):
#         row = df.iloc[i]
#         for p, c in zip(row[:-1], row[1:]):
#             edge = pydot.Edge(p, c)
#             graph.add_edge(edge)
#     graph.write_png(f"img/flowcharts/situ_cluster_{cluster}.png")

In [ ]:
# nrows, ncols = df2.shape
# for i in range(nrows):
#     row = df2.iloc[i]
#     for p, c in zip(row[:-1], row[1:]):
#         edge = pydot.Edge(p, c)
#         graph.add_edge(edge)

In [ ]:
# graph.write_png('img/situ_clusters/graph_cluster1.png')

In [ ]:
# for df in df2.groupby('cluster'):
#     print(df)

In [ ]:
import flask
